# CoronaVirus

* Coronavirus disease (COVID-19) is an infectious disease caused by a newly discovered coronavirus.

* Most people infected with the COVID-19 virus will experience mild to moderate respiratory illness and recover without requiring special treatment.  Older people, and those with underlying medical problems like cardiovascular disease, diabetes, chronic respiratory disease, and cancer are more likely to develop serious illness.
 
* The best way to prevent and slow down transmission is to be well informed about the COVID-19 virus, the disease it causes and how it spreads. Protect yourself and others from infection by washing your hands or using an alcohol based rub frequently and not touching your face. 
 
* The COVID-19 virus spreads primarily through droplets of saliva or discharge from the nose when an infected person coughs or sneezes, so it’s important that you also practice respiratory etiquette (for example, by coughing into a flexed elbow).


<img src = "https://i.pinimg.com/originals/a8/d8/66/a8d866c87e75ae65af8a3f167ee1e8f6.gif"/>

In [119]:
#Import pandas to convert list to data frame

import pandas as pd
import numpy as np
import requests
from pandas_profiling import ProfileReport

import urllib.request

#import the beatiful soup functions to parse the data
from bs4 import BeautifulSoup


from plotly.offline import iplot, init_notebook_mode
import plotly.express as px
import plotly.graph_objects as go

import cufflinks as cf
cf.go_offline()
import os

# Any results you write to the current directory are saved as output.
from IPython.display import display, Image


import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point,Polygon

import math

In [120]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/alldatas/kiXT+ZIB
/kaggle/input/alldatas/total_deaths.csv
/kaggle/input/alldatas/total_cases.csv
/kaggle/input/alldatas/locations.csv
/kaggle/input/alldatas/new_deaths.csv
/kaggle/input/alldatas/owid-covid-data.csv
/kaggle/input/alldatas/new_cases.csv
/kaggle/input/alldatas/full_data.csv


# Importing and Visualization of data

In [149]:
full_data = pd.read_csv('/kaggle/input/alldatas/full_data.csv')

full_data.head()

,date,location,new_cases,new_deaths,total_cases,total_deaths,weekly_cases,weekly_deaths,biweekly_cases,biweekly_deaths
0,2019-12-31,Afghanistan,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,2020-01-01,Afghanistan,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,2020-01-02,Afghanistan,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,2020-01-03,Afghanistan,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,2020-01-04,Afghanistan,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


# Substitute Nan with zeros

In [150]:
full_data = full_data.fillna(0)

full_data.head()

,date,location,new_cases,new_deaths,total_cases,total_deaths,weekly_cases,weekly_deaths,biweekly_cases,biweekly_deaths
0,2019-12-31,Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-01,Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-02,Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-03,Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-04,Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Providing datatype to each column

In [125]:
full_data.columns

Index(['date', 'location', 'new_cases', 'new_deaths', 'total_cases',
       'total_deaths', 'weekly_cases', 'weekly_deaths', 'biweekly_cases',
       'biweekly_deaths'],
      dtype='object')

In [151]:
from datetime import datetime

full_data = full_data.astype({
    'location':str, 
    'new_cases':int, 
    'new_deaths':int,
    'total_cases':int,
    'total_deaths':int,
    'weekly_cases':int,
    'weekly_deaths':int,
    'biweekly_cases':int,
    'biweekly_deaths':int
})


full_data.date = pd.to_datetime(full_data.date)

In [152]:
full_data.dtypes

date               datetime64[ns]
location                   object
new_cases                   int64
new_deaths                  int64
total_cases                 int64
total_deaths                int64
weekly_cases                int64
weekly_deaths               int64
biweekly_cases              int64
biweekly_deaths             int64
dtype: object

# Plotting the graph of total number of cases till date

In [174]:
# Extracting required features and processing time

full_data_total = full_data[['date','location','total_cases', 'total_deaths']]

latest_time = pd.to_datetime('2020-10-11 00:00:00')

def return_nepal(date):
    if(date == latest_time):
        return True
    else:
        return False
    
def return_noworld(location):
    if(location!='World'):
        return True
    else:
        return False
    
full_data_total = full_data_total[full_data_total['date'].apply(return_nepal)]
full_data_total = full_data_total[full_data_total['location'].apply(return_noworld)]

full_data_total_table = full_data_total[['location','total_cases', 'total_deaths']]
full_data_total_table = full_data_total_table.sort_values('total_cases', ascending=False)

full_data_total_table = full_data_total_table.reset_index(drop=True)
full_data_total_table.style.background_gradient(cmap='Oranges')

,location,total_cases,total_deaths
0,United States,7718947,214377
1,India,7053806,108334
2,Brazil,5082637,150198
3,Russia,1285084,22454
4,Colombia,902747,27660
5,Argentina,883883,23581
6,Peru,846088,33223
7,Mexico,814328,83642
8,France,718873,32684
9,South Africa,690896,17673


In [131]:
# Visualizing the data

from plotly.subplots import make_subplots

labels = full_data_total['location'].tolist()

values1 = full_data_total['total_cases'].tolist()
values2 = full_data_total['total_deaths'].tolist()

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

fig.add_trace(go.Pie(labels=labels, values=values1, name="Total Cases"),
              1, 1)
fig.add_trace(go.Pie(labels=labels, values=values2, name="Total Deaths"),
              1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent+name", textposition='inside')

fig.update_layout(
    autosize=True,
    title_text="CoronaVirus Total Cases and Death Cases per Country",
    # Add annotations in the center of the donut pies.
    annotations=[
                 dict(text='Total Cases', x=0.2, y=0.5, font_size=13, showarrow=False),
                dict(text='Total Deaths', x=0.8, y=0.5, font_size=13, showarrow=False)])
fig.show()

# Visualizing Total Cases in Map

In [132]:
trace1 = [go.Choropleth(
               colorscale = 'Picnic',
               locationmode = 'country names',
               locations = labels,
               text = labels, 
               z = values1,
               )]

layout = dict(title = 'CoronaVirus Total Cases',
                  geo = dict(
                      showframe = True,
                      showocean = True,
                      showlakes = True,
                      showcoastlines = True,
                      projection = dict(
                          type = 'hammer'
        )))


projections = [ "equirectangular", "mercator", "orthographic", "natural earth","kavrayskiy7", 
               "miller", "robinson", "eckert4", "azimuthal equal area","azimuthal equidistant", 
               "conic equal area", "conic conformal", "conic equidistant", "gnomonic", "stereographic", 
               "mollweide", "hammer", "transverse mercator", "albers usa", "winkel tripel" ]

buttons = [dict(args = ['geo.projection.type', y],
           label = y, method = 'relayout') for y in projections]
annot = list([ dict( x=0.1, y=0.8, text='Projection', yanchor='bottom', 
                    xref='paper', xanchor='right', showarrow=False )])


# Update Layout Object

layout[ 'updatemenus' ] = list([ dict( x=0.1, y=0.8, buttons=buttons, yanchor='top' )])
layout[ 'annotations' ] = annot


fig = go.Figure(data = trace1, layout = layout)
iplot(fig)


# Bar Chart : Total Cases and Death Cases

In [148]:
full_data_total_sort = full_data_total.sort_values('total_cases', ascending=False)
full_data_total_10 = full_data_total_sort[:10]

full_data_total_10.iplot(x='location', y=['total_cases', 'total_deaths'], kind='bar', xTitle='Country', yTitle='Number of Cases', title='Top 10 Country with CoronaVirus Cases and Death Cases')

# Interesting Facts

1. Top country having highest cases USA, India and Brazil
2. USA has the highest Corona cases and death cases
3. Even having less number of Corona Cases than India, Brazil has more death cases

# Plotting only the graph of Nepal(TimeSeries)

In [135]:
# Extracting only data of Nepal
def return_nepal(location):
    if(location in ['Nepal']):
        return True
    else:
        return False

full_data_nepal = full_data[full_data.location.apply(return_nepal)]



# Visualizing the time Series data 
fig = go.Figure()
fig.add_trace(go.Scatter(x=full_data_nepal['date'], y=full_data_nepal['total_cases'], fill='tozeroy',name='Total Cases'))
fig.add_trace(go.Scatter(x=full_data_nepal['date'], y=full_data_nepal['total_deaths'], fill='tozeroy',name='Total Death'))


# Set x-axis title
fig.update_xaxes(title_text="Year")
fig.update_yaxes(title_text="Number of Cases")
fig.show()

# Gaining more insight using BoxPlot

In [158]:
import plotly.express as px 

#New CoronaVirus Cases 
fig = px.box(full_data_nepal, y='new_cases', title='CoronaVirus New Cases Per Day')
fig.show()

In [160]:
#New CoronaVirus Death Cases 
fig = px.box(full_data_nepal, y='new_deaths', title='CoronaVirus New Death Cases Per Day')
fig.show()

# Facts

1. Highest CoronaVirus Cases in a Day is 5008
1. Highest CoronaVirus Death Cases in a Day is 19